## Notebook to test the performance of the IterativeFitter class in case a variable background is present on top of the signal

In [ ]:
import pandas as pd
import numpy as np
from ramp_utils.ramp import RampTimeSeq,RampMeasurement
from ramp_utils.fitter import IterativeFitter
import matplotlib.pyplot as plt
import time, sys, pickle, bz2
from multiprocessing import Pool
from scipy.interpolate import interp1d
import copy

%matplotlib notebook

### First define a ramp time sequence and a measurement

In [ ]:
myramp = RampTimeSeq('HST/WFC3/IR',15,samp_seq='SPARS100') 
myflux = 1.
CRdict = None  # For no cosmic rays
mymeas = RampMeasurement(myramp,myflux,CRdict=CRdict)  #an example for WFC3 ramps

### Run the IterativeFitter on it (no extra background added)

In [ ]:
myramp.test_plot()

mymeas.test_plot()

myfitter = IterativeFitter(mymeas)
err, count, gi, crlcount = myfitter.perform_fit(CRthr=4)
myfitter.test_plot()
myfitter.goodness_of_fit(mode='Squared-deviations')

print('Error message',err)
print('Iterations',count)
print('fitted flux (e/s)', myfitter.mean_electron_rate)
print('real flux (e/s)',myfitter.RM.flux)
print(gi)
print('Iterations for CR rej',crlcount)
print('GOF-stat,GOF-p',myfitter.gof_stat,myfitter.gof_pval)

### Define a variable background within the ramp's time interval. Scale it by a fixed average amount (in terms of electrons/s)

In [ ]:
bg_times      = np.linspace(0,1400,10)

#bg_countrates = np.array([1.1,1.5,2,6,3,2.5,1.5])
#bg_countrates = np.array([1.1,1.5,2,3.1,3,2.5,1.5])
#bg_countrates = np.array([1.,1.5,2,2.5,3,3.5,4])
bg_electron_rate = np.array([1.,1.5,1.6,1.4,1.7,2.2,2.4])
bg_electron_rate = np.array([1.,1.2,1.4,1.3,1.7,2.,2.2,2.4,2.0,1.5])
mean_bg_electron_rate = 1.5

bg_int = interp1d(bg_times,bg_electron_rate,'quadratic')
varbg = bg_int(myramp.read_times)

dt = myramp.read_times[-1]-myramp.read_times[0]
t_avg = np.trapz(varbg,myramp.read_times) / dt
varbg = varbg/t_avg * mean_bg_electron_rate
a_rms = np.sqrt(np.trapz(np.square(varbg-mean_bg_electron_rate),myramp.read_times)/dt)

f = plt.figure()
ax = f.add_subplot(111)
ax.set_title('Background rate (e/s), mean = {:7.4f}, rms ={:7.4f}'.format(mean_bg_electron_rate,a_rms))
ax.set_xlabel('Time (s)')
ax.set_ylabel('Electrons/s')
ax.plot(myramp.read_times,varbg);

### Generate variable background cumulated electrons from the interpolated countrate

In [ ]:
bg_electrons=[0]
bg_electrons.extend([np.random.poisson(lam=vb*dt) for vb,dt in zip(0.5*(varbg[1:]+varbg[:-1]),mymeas.RTS.read_times[1:]-mymeas.RTS.read_times[:-1]) ])
bg_electrons = np.asarray(bg_electrons)
print(bg_electrons)
print(np.cumsum(bg_electrons))

### Copy the previous measurement and add the variable backgoround

In [ ]:
mymeas2 = copy.copy(mymeas)
mymeas2.add_background(np.cumsum(bg_electrons))

### Run fitter on the ramp with extra background

In [ ]:
myramp.test_plot()

mymeas2.test_plot()

myfitter2 = IterativeFitter(mymeas2)
err2, count2, gi2, crlcount2 = myfitter2.perform_fit(CRthr=4)
myfitter2.test_plot()
myfitter2.goodness_of_fit(mode='Pearson-chi-sq')

print('Error message',err2)
print('Iterations',count2)
print('fitted flux (e-/s)', myfitter2.mean_electron_rate)
print('real flux (e-/s)',myfitter2.RM.flux)
print(gi2)
print('Iterations for CR rej',crlcount2)
print('GOF-stat,GOF-p',myfitter2.gof_stat,myfitter2.gof_pval)